In [8]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise import Trainset
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

from collections import defaultdict
from operator import itemgetter
import heapq

import os
import csv

import pandas as pd

In [2]:
UsersDF = pd.read_csv('../raw_data/users_cleaned.csv')
AnimesDF = pd.read_csv('../raw_data/anime_cleaned.csv')
ScoresDF = pd.read_csv('../raw_data/animelists_cleaned.csv')

In [4]:
ScoresDF_selected= ScoresDF[ScoresDF["my_score"]>0][["username","anime_id","my_score","my_last_updated"]]

In [22]:
small_fraction = 0.001
ScoresDF_selected_small = ScoresDF_selected.sample(int(len(ScoresDF_selected)*small_fraction))
ScoresDF_selected_small.shape

(19172, 4)

In [23]:
reader = Reader(rating_scale=(0, 10))
scoredata = Dataset.load_from_df(ScoresDF_selected_small[['username', 'anime_id', 'my_score']], reader)
# trainset = scoredata.build_full_trainset()

In [24]:
trainset, testset = train_test_split(scoredata, test_size=0.25)

In [35]:
from surprise.model_selection import cross_validate

In [25]:
knn_predictor = KNNBasic(sim_options={
        'name': 'cosine',
        'user_based': False
        })
        
knn_predictor.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [26]:
predictions = knn_predictor.test(testset)

In [27]:
accuracy.rmse(predictions)

RMSE: 1.6924


1.6923991808036007

In [28]:
svd_predictor = SVD()

In [29]:
svd_predictor.fit(trainset)

In [30]:
svd_predictions = svd_predictor.test(testset)

In [31]:
accuracy.rmse(svd_predictions)

RMSE: 1.6131


1.6131398682788674

In [32]:
similarity_matrix = svd_predictor.compute_similarities()

In [33]:
similarity_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [39]:
from surprise import accuracy

In [43]:
similarity_matrix.shape

(12460, 12460)

In [40]:
k=10
# Generate predictions for the test set
predictions_svd = svd_predictor.test(testset)

# Compute precision at K
precision_svd = accuracy.precision_at_k(predictions_svd, k, threshold=8.0)
print("Precision at K:", precision_svd)

AttributeError: module 'surprise.accuracy' has no attribute 'precision_at_k'

In [42]:
k=10
# Compute precision at K
precision = 0
for uid in trainset.all_users():
    true_items = [iid for (iid, _) in testset if iid in trainset.ur[uid]]
    if len(true_items) > 0:
        user_ratings = svd_predictor.get_neighbors(uid, k=k)
        top_k_items = [iid for (iid, _) in user_ratings]
        precision += len(set(true_items) & set(top_k_items)) / k

precision /= trainset.n_users

print("Precision at K:", precision)

ValueError: too many values to unpack (expected 2)

In [ ]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin("ml-100k")

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)